# PDF 에서 introduction, related works, background 추출

In [5]:
from datetime import datetime
import pickle
import re
import os
import fitz  # PyMuPDF
import pandas as pd
from tqdm import tqdm

from IPython.display import clear_output
import logging

### load data

In [ ]:
with open ("../data_files/paper_metadata/arxiv_ascending_20161001_20230920.pkl", 'rb') as f:
    paper_meta = pickle.load(f)
    
with open ("../data_files/paper_metadata/arxiv_descending_2024_20160927.pkl", 'rb') as f:
    after2023 = pickle.load(f)

paper_meta.extend(after2023)
paper_total = sorted(paper_meta, key=lambda x: x['published'])

### pdf 에서 필요한 부분 (intro, related_works, background 추출)

#### PyMuPDF 사용해서 논문의 섹션을 인식 후, 필요한 파트 추출

In [6]:
# 추가 처리해야하는 사항들
# 각주가 있어서 section으로 처리되는 경우 
# Introduction, Method와 같이 맨 앞에 숫자가 없는 경우
# Introduction, Method와 같이 맨 앞에 로마 숫자인 경우 (해결)

In [3]:
def sanitize_title(title):
    title_fixed = title.replace('\n', ' ').strip()  # 줄바꿈 제거 및 양끝 공백 제거
    title_fixed = re.sub(r'\s+', ' ', title_fixed)  # 여러 공백을 하나로 축소
    title_fixed = title_fixed.replace('/', '_')  # '/'를 '_'로 대체
    return title_fixed

def is_page_number(text): # 페이지 번호도 section으로 취급되는 것을 막기 위해 추가한 함수
    """
    텍스트가 페이지 번호인지 확인하는 함수.
    - 단일 숫자 또는 "Page X" 형식 등으로 페이지 번호를 판단.
    """
    text = text.strip().lower()
    # 단일 숫자 또는 "page X" 형식의 텍스트인지 확인
    if text.isdigit():  # 숫자로만 구성된 경우
        return True
    if text.startswith("page") and text[4:].strip().isdigit():  # "Page 1", "page 2" 등
        return True
    return False

roman_numeral_pattern = re.compile(r"^(?:[IVXLCDM]+)\.? ")  # 로마 숫자로 시작, 뒤에 . 또는 공백
alphabetic_subheading_pattern = re.compile(r"^[A-Z]\. ")     # 알파벳과 점으로 시작 (A., B., C. 등)

def extract_sections(file_path):
    doc = fitz.open(file_path)
    sections = []
    current_section = None
    current_text = ""

    for page in doc:
        blocks = page.get_text("blocks")  # 페이지 내 텍스트 블록 가져오기
        for block in blocks:
            text = block[4].strip()  # 블록 텍스트 추출
            #print(text)
            
            # 페이지 번호인지 확인 
            if is_page_number(text):
                continue
            
            # 섹션 제목 탐지 (섹션 번호로 시작하는 경우)
            if text.startswith(tuple(str(i) for i in range(1, 10))) or roman_numeral_pattern.match(text):  # "1", "2" 등으로 시작
                if current_section:  # 현재 섹션 저장
                    sections.append((current_section, current_text.strip()))
                current_section = text  # 새로운 섹션 제목
                #print(current_section)
                current_text = ""  # 섹션 텍스트 초기화
                
            # 2. 알파벳 소제목 감지 (A., B., C. 등)
            elif alphabetic_subheading_pattern.match(text):
                current_text += f"\n{text}\n"  # 기존 섹션 본문에 추가
            
            else:
                current_text += f"{text} "  # 섹션 본문 추가

    # 마지막 섹션 추가
    if current_section:
        sections.append((current_section, current_text.strip()))

    return sections

In [4]:
# 하위 섹션 여부를 판단하는 함수
def is_subsection_of(parent, child):
    """
    주어진 `child` 섹션이 `parent` 섹션의 하위인지 확인.
    숫자와 로마 숫자 모두 처리.
    """
    if parent.isdigit():  # 숫자 섹션 (예: 2.1, 2.2)
        return child.startswith(parent)
    else:  # 로마 숫자 섹션 (예: II.A, II.B)
        return child.split(".")[0] == parent

# introduction, related_works, background 추출
def extract_specific_section(file_path, target_section):
    """
    특정 섹션과 그 하위 내용을 추출.
    - 타겟 섹션 번호 아래 A., B., C. 같은 알파벳 소제목 포함.
    """
    # 타겟 섹션 이름과 대소문자 및 복수형을 포함해 매칭할 정규 표현식 생성
    target_section_pattern = re.compile(
        rf"{re.escape(target_section)}(s)?", re.IGNORECASE
    )

    # 숫자 및 로마 숫자 섹션 추출 정규식
    section_num_pattern = re.compile(r"^(\d+|I{1,3}|IV|V|VI|VII|VIII|IX|X)(\b|\.|\s)")
    
    # 알파벳 소제목(A., B., ...) 정규식
    alphabetic_subheading_pattern = re.compile(r"^[A-Z]\.\s")

    sections = extract_sections(file_path)
    extracted_text = ""
    start_section_num = None
    current_major_section = None  # 현재 주요 섹션 번호 저장

    for section_title, section_text in sections:
        # 숫자 또는 로마 숫자로 시작하는 섹션 번호 추출
        section_num_match = section_num_pattern.match(section_title)
        section_num = section_num_match.group(1) if section_num_match else None
        
        # 1. 타겟 섹션 시작 감지
        if start_section_num is None and target_section_pattern.search(section_title):
            start_section_num = section_num  # 타겟 섹션 번호 저장 (예: II)
            current_major_section = section_num
            extracted_text += f"{section_title}\n{section_text}\n"

        # 2. 타겟 섹션 번호 또는 하위 섹션 포함 (예: II.A, II.B)
        # section_num.startswith(start_section_num) 이걸로 하면 III 은 II로 시작하는게 맞아서 수정되어버림 아오ㅓ
        elif start_section_num and section_num and is_subsection_of(start_section_num, section_num):
            current_major_section = section_num.split(".")[0]  # 주요 섹션 번호 업데이트
            extracted_text += f"{section_title}\n{section_text}\n"

        # 3. 알파벳 소제목(A., B., C.) 포함 (현재 주요 섹션에 속하는 경우만)
        elif current_major_section and alphabetic_subheading_pattern.match(section_title):
            extracted_text += f"{section_title}\n{section_text}\n"

        # 4. 새로운 주요 섹션으로 넘어가면 종료
        elif start_section_num and section_num and start_section_num != section_num:
            break

    return extracted_text if extracted_text else f"CODE998825"

In [7]:
# 에러 로그 저장
logging.basicConfig(
    filename="processing_pdf_after_error_intro.log",  # 로그 파일 이름
    filemode="w",                  # 로그 파일 덮어쓰기
    level=logging.INFO,            # 로그 레벨 설정 (INFO 이상)
    format="%(asctime)s - %(levelname)s - %(message)s"
)

data = []

def is_valid_pdf(file_path):
    """PDF 파일 유효성 검사."""
    try:
        with fitz.open(file_path) as doc:
            return True
    except Exception as e:
        logging.error(f"Invalid PDF: {file_path} - Error: {e}")
        return False

# 처리 진행
for idx, paper in enumerate(tqdm(paper_total, desc="Processing Papers")):
    try: 
        entry_id = paper['entry_id'].split('/')[-1]
        title = sanitize_title(paper['title'])
        author = ', '.join(paper['authors'])
        
        published = datetime.strftime(paper['published'], format='%Y-%m-%d')
        published_year = published.split('-')[0]
        published_month = published.split('-')[1]
        published_day = published.split('-')[2]
        
        abstract = paper['summary']
        paper_link = paper['links'][1]
        pdf_file_path = f'../paper_연도별/paper_{published_year}/{title}.pdf'
        
        if not os.path.exists(pdf_file_path) or not is_valid_pdf(pdf_file_path):
            logging.warning(f"Skipping invalid or missing PDF: {pdf_file_path}")
            introduction = "CODE 997725"
            related_work = "CODE 997725"
            background = "CODE 997725"
            
        else:
            try:
                introduction = extract_specific_section(pdf_file_path, 'Introduction')
            except Exception as e:
                logging.error(f"Error extracting 'Introduction' in '{title}' (Path: {pdf_file_path}): {e}")
                introduction = "CODE 997725"
            
            try:
                related_work = extract_specific_section(pdf_file_path, 'Related Work')
            except Exception as e:
                logging.error(f"Error extracting 'Related Work' in '{title}' (Path: {pdf_file_path}): {e}")
                related_work = "CODE 997725"

            try:
                background = extract_specific_section(pdf_file_path, 'Background')
            except Exception as e:
                logging.error(f"Error extracting 'Background' in '{title}' (Path: {pdf_file_path}): {e}")
                background = "CODE 997725"
        
        # 데이터를 리스트로 추가
        data.append({
            "entry_id": entry_id,
            "title": title,
            "author": author,
            "published": published,
            "published_year": published_year,
            "published_month": published_month,
            "published_day": published_day,
            "abstract": abstract,
            "introduction": introduction,
            "paper_link": paper_link,
            "related_work": related_work,
            "background": background
        })
        
    except Exception as e:
        logging.error(f"Error processing paper with entry_id {paper['entry_id']}: {e}")
        continue
    
    # 매 200개마다 출력 초기화
    if (idx + 1) % 200 == 0:
        clear_output(wait=True)  # 이전 출력 삭제
        print(f"Processed {idx + 1} papers so far...")  # 진행 상황 표시
    
    # 매 10000개마다 백업 저장
    if (idx + 1) % 10000 == 0:
        df = pd.DataFrame(data)
        backup_filename = f"../data_files/filtered_data/backup_data/processed_papers_backup_end_{idx + 1}.parquet"
        df.to_parquet(backup_filename, index=False)
        logging.info(f"Backup saved at {backup_filename}")

# 최종 데이터프레임 저장
df = pd.DataFrame(data)
df.to_parquet("../data_files/filtered_data/processed_papers.parquet", index=False)
logging.info("Final processed papers saved.")


Processed 70000 papers so far...


Processing Papers: 100%|██████████| 70000/70000 [4:44:12<00:00,  4.11it/s]  
